Universidad Del Valle de Guatemala  
Departamento de Computación  
Minería de datos  

Proyecto - Análisis exploratorio  
José Pablo Kiesling Lange - 21581  
Erick Stiv Junior Guerra Muñoz - 21781  
Diego Andrés Morales Aquino - 21762  
Pablo Andrés Zamora Vásquez - 21780

In [118]:
# Importando librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [119]:
# Leyendo archivo
datos = pd.read_csv('SDGData.csv')

In [120]:
datos.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1990,1991,1992,1993,1994,1995,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 35
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,85.644218,85.932567,86.232384,86.478597,86.722685,86.937933,87.040774,87.235539,87.307068,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,87.039588,88.992620,88.015356,88.681886,89.195062,90.324659,88.910749,89.999946,90.277735,NaN
2,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,54.722944,56.807039,57.872605,57.447725,...,75.377022,79.582622,77.666157,78.951592,79.791665,82.373066,82.970754,83.740500,81.660899,NaN
3,Arab World,ARB,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,96.980079,97.239057,96.856101,97.021313,97.261799,97.483658,97.581237,98.278287,97.540397,NaN
4,Arab World,ARB,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,30.470000,NaN,NaN,37.230000,NaN,NaN,NaN,NaN


### Parte 1: Explore los datos para encontrar preguntas interesantes y líneas de investigación. Para esto:

a. Comience describiendo cuántas variables y observaciones tiene disponibles, y el tipo de cada una de las variables.

In [121]:
num_rows, num_columns = datos.shape

print("Observaciones:", num_rows)
print("Variables:", num_columns)

Observaciones: 106488
Variables: 36


In [122]:
# Tipo de dato de cada columna
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106488 entries, 0 to 106487
Data columns (total 36 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Country Name    106488 non-null  object 
 1   Country Code    106488 non-null  object 
 2   Indicator Name  106488 non-null  object 
 3   Indicator Code  106488 non-null  object 
 4   1990            23997 non-null   float64
 5   1991            28622 non-null   float64
 6   1992            30313 non-null   float64
 7   1993            29447 non-null   float64
 8   1994            29893 non-null   float64
 9   1995            32175 non-null   float64
 10  1996            31622 non-null   float64
 11  1997            33001 non-null   float64
 12  1998            32370 non-null   float64
 13  1999            34820 non-null   float64
 14  2000            45687 non-null   float64
 15  2001            42846 non-null   float64
 16  2002            44840 non-null   float64
 17  2003      

In [123]:
# Cantidad de indicadores distintos
datos["Indicator Name"].nunique()

407

In [124]:
# Eliminar columna no útil
datos = datos.drop(columns=["Unnamed: 35"])
datos = datos.drop(columns=["1990"])
datos = datos.drop(columns=["1991"])
datos = datos.drop(columns=["1992"])
datos = datos.drop(columns=["1993"])
datos = datos.drop(columns=["1994"])
datos = datos.drop(columns=["1995"])
datos = datos.drop(columns=["1996"])
datos = datos.drop(columns=["1997"])
datos = datos.drop(columns=["1998"])
datos = datos.drop(columns=["1999"])
datos = datos.drop(columns=["2000"])
datos = datos.drop(columns=["2001"])
datos = datos.drop(columns=["2002"])
datos = datos.drop(columns=["2003"])
datos = datos.drop(columns=["2004"])

In [126]:
# Transformar data
melted = pd.melt(datos, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='Year', value_name='Value')
pivoted = melted.pivot_table(index=['Country Name', 'Year'], columns='Indicator Name', values='Value', aggfunc="mean").reset_index()

tData = pivoted


Indicator Name,Country Name,Year,Access to clean fuels and technologies for cooking (% of population),Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",Account ownership at a financial institution or with a mobile-money-service provider (% of population ages 15+),"Account ownership at a financial institution or with a mobile-money-service provider, female (% of population ages 15+)","Account ownership at a financial institution or with a mobile-money-service provider, male (% of population ages 15+)","Account ownership at a financial institution or with a mobile-money-service provider, older adults (% of population ages 25+)",...,Urban population growth (annual %),"Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal)",Women Business and the Law Index Score (scale 1-100),"Women making their own informed decisions regarding sexual relations, contraceptive use and reproductive health care (% of women age 15-49)",Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),tax revenue (current LCU)
0,Afghanistan,2005,12.20,25.390894,11.113852,74.000000,NaN,NaN,NaN,NaN,...,4.580880,1.48,9.00,7.86,NaN,26.25,NaN,NaN,NaN,NaN
1,Afghanistan,2006,13.85,30.718691,15.765366,81.043770,NaN,NaN,NaN,NaN,...,3.885074,1.54,9.24,8.08,NaN,26.25,NaN,NaN,NaN,2.425061e+10
2,Afghanistan,2007,15.30,36.051010,22.060678,82.590820,NaN,NaN,NaN,NaN,...,3.389088,1.63,9.88,8.65,0.54593,28.75,NaN,NaN,NaN,2.564691e+10
3,Afghanistan,2008,16.70,42.400002,27.954250,89.900002,NaN,NaN,NaN,NaN,...,3.159892,1.70,10.18,8.92,NaN,28.75,NaN,NaN,39.0,3.135587e+10
4,Afghanistan,2009,18.40,46.740051,34.745785,85.724472,NaN,NaN,NaN,NaN,...,3.284962,2.05,11.56,10.15,NaN,28.75,NaN,NaN,NaN,5.188225e+10


In [129]:
cleanedCols = tData.select_dtypes(include='number').columns
tData = tData.fillna(tData.mean())
tData.head()

TypeError: Could not convert ['200520062007200820092010201120122013201420152016201720182019202020052006200720082009201020112012201320142015201620172018201920202005200620072008200920102011201220132014201520162017201820192020200520062007200820092010201120122013201420152016201720182019202020052006200720082009201020112012201320142015201620172018201920202005200620072008200920102011201220132014201520162017201820192020200520062007200820092010201120122013201420152016201720182019202020052006200720082009201020112012201320142015201620172018201920202005200620072008200920102011201220132014201520162017201820192020200520062007200820092010201120122013201420152016201720182019202020052006200720082009201020112012201320142015201620172018201920202005200620072008200920102011201220132014201520162017201820192020200520062007200820092010201120122013201420152016201720182019202020052006200720082009201020112012201320142015201620172018201920202005200620072008200920102011201220132014201520162017201820192020200520062

In [127]:
tData.shape

(4176, 405)